**Importacion de las librerias**


In [11]:
import pandas as pd
import matplotlib

import numpy as np
import math
import datetime

Funcion generadora de ruido gassiano utilizando le metodo de Box-Muller Transformation

In [12]:
def obtener_segundos_actual():
    # Obtener la hora actual
    hora_actual = datetime.datetime.now()
    
    # Extraer los segundos de la hora actual
    segundos = hora_actual.second
    
    return segundos

In [13]:
def obtener_minutos_actual():
    # Obtener la hora actual
    hora_actual = datetime.datetime.now()
    
    # Extraer los minutos de la hora actual
    minutos = hora_actual.minute
    
    return minutos

In [14]:
def obtener_hora_actual():
    # Obtener la hora actual
    hora_actual = datetime.datetime.now()
    
    # Extraer solo la hora actual
    hora = hora_actual.hour
    
    return hora


In [15]:


PI = 3.1415926536

def AWGN_generator():
    """Genera muestras de ruido gaussiano blanco con media cero y desviación estándar de uno."""
    while True:
        temp2 = np.random.rand()
        if temp2 != 0:
            break

    temp1 = np.cos(2.0 * PI * np.random.rand())
    result = math.sqrt(-2.0 * math.log(temp2)) * temp1

    # Escala el resultado al rango [-1, 1]
    result = np.clip(result, -1, 1)
    
    return result


In [16]:
# funcion aleatorizadora de semilla
def random_semilla ():
    np.random.seed(int(obtener_segundos_actual()*obtener_minutos_actual()*obtener_hora_actual()))
    

In [17]:
def lector_csv():
  
    import pandas as pd

    # Nombre del archivo CSV
    archivo_csv = 'modulado.txt'
    
    # Cargar el archivo CSV en un DataFrame de Pandas
    datos = pd.read_csv(archivo_csv, header=None)
    
    # Cambiar la letra 'i' por 'j' en cada dato y convertirlos en números complejos
    datos[0] = datos[0].str.replace('i', 'j')
    datos[0] = datos[0].apply(lambda x: complex(x))
    
    # Mostrar el DataFrame resultante
    return datos
    
   

In [18]:
def agregar_ruido(datos):
    # Iterar sobre las filas del DataFrame
    for indice_fila, fila in datos.iterrows():
        # Iterar sobre los elementos de la fila
        random_semilla ()
        for indice_elemento, elemento in enumerate(fila):
            random_semilla ()
            # Separar la parte real e imaginaria del número complejo
            parte_real, parte_imaginaria = np.real(elemento), np.imag(elemento)
            # Generar ruido para la parte real e imaginaria
            ruido_real, ruido_imaginario = AWGN_generator(), AWGN_generator()
            # Agregar el ruido a la parte real e imaginaria
            datos.iloc[indice_fila, indice_elemento] = parte_real + ruido_real + 1j * (parte_imaginaria + ruido_imaginario)
    return datos

In [19]:
# Función para agregar ruido a los datos del DataFrame


# Llamar a las funciones
data_frame = lector_csv()
datos_con_ruido = agregar_ruido(data_frame)

datos_con_ruido[0] = datos_con_ruido[0].apply(lambda x: str(x))
datos_con_ruido[0] = datos_con_ruido[0].str.replace('j', 'i')


datos_con_ruido.to_csv('salida_canal.txt', index=False, header=None)

# Imprimir un mensaje de confirmación
print("Los datos con ruido se han guardado en el archivo 'datos_con_ruido.csv'.")


Los datos con ruido se han guardado en el archivo 'datos_con_ruido.csv'.
